In [4]:
import mne
import numpy as np

# Load a single run file
fif_path = "/srv/synaptech_openfmri/test/sub-06/run_03.fif"

# Load the raw data
raw = mne.io.read_raw_fif(fif_path, preload=True)

# Get basic info
print("\nFile Info:")
print(f"Path: {fif_path}")
print(f"Number of channels: {len(raw.ch_names)}")
print(f"Sampling frequency: {raw.info['sfreq']} Hz")
print(f"Duration: {raw.times.max():.2f} seconds")

# Get the data array
data = raw.get_data()
print("\nData Shape:")
print(f"Shape: {data.shape}")  # Should be (n_channels, n_timepoints)
print(f"Data type: {data.dtype}")

# Basic statistics
print("\nData Statistics:")
print(f"Mean: {np.mean(data):.4f}")
print(f"Std: {np.std(data):.4f}")
print(f"Min: {np.min(data):.4f}")
print(f"Max: {np.max(data):.4f}")

# Show first few channel names
print("\nFirst 5 channels:")
print(raw.ch_names[:5])

# Also check if there's a corresponding .txt file
import os
txt_path = fif_path.replace('.fif', '.txt')
if os.path.exists(txt_path):
    print("\nCorresponding .txt file exists:")
    with open(txt_path, 'r') as f:
        first_line = f.readline().strip()
    print(f"First line: {first_line}")


File Info:
Path: /srv/synaptech_openfmri/test/sub-06/run_03.fif
Number of channels: 404
Sampling frequency: 220.0 Hz
Duration: 497.50 seconds

Data Shape:
Shape: (404, 109450)
Data type: float64

Data Statistics:
Mean: 19.0903
Std: 387.2113
Min: -9.9427
Max: 7936.0000

First 5 channels:
['MEG0113', 'MEG0112', 'MEG0111', 'MEG0122', 'MEG0123']

Corresponding .txt file exists:
First line: ************************************************************


- mean-pooling

In [2]:
raw = mne.io.read_raw_fif("/home/gleb/Synaptech/data_magic/data/openfmri/train/sub-05/run_03.fif")

/tmp/ipykernel_303605/1096166328.py:1: RuntimeWarning: Invalid tag with only 0/16 bytes at position 890435679 in file /home/gleb/Synaptech/data_magic/data/openfmri/train/sub-05/run_03.fif
  raw = mne.io.read_raw_fif("/home/gleb/Synaptech/data_magic/data/openfmri/train/sub-05/run_03.fif")
/tmp/ipykernel_303605/1096166328.py:1: RuntimeWarning: FIF tag directory missing at the end of the file (at byte 890435679 > file size 304070656), possibly corrupted file: /home/gleb/Synaptech/data_magic/data/openfmri/train/sub-05/run_03.fif
  raw = mne.io.read_raw_fif("/home/gleb/Synaptech/data_magic/data/openfmri/train/sub-05/run_03.fif")
/tmp/ipykernel_303605/1096166328.py:1: RuntimeWarning: Invalid tag with only 0/16 bytes at position 305599959 in file /home/gleb/Synaptech/data_magic/data/openfmri/train/sub-05/run_03.fif
  raw = mne.io.read_raw_fif("/home/gleb/Synaptech/data_magic/data/openfmri/train/sub-05/run_03.fif")


In [13]:
from torch.utils.data import Dataset
import os
import logging
import traceback
import shutil
import zipfile
import random
import re
import numpy as np
import contextlib
import mne
from tqdm import tqdm  # for progress bars
import warnings
import torch
import warnings

def  _meanPoolData(datasetPath):
    pooled_marker = os.path.join(datasetPath, '.pooled')
    if os.path.exists(pooled_marker):
        return
            
    warnings.filterwarnings("ignore",message=".* does not conform to MNE naming conventions.*",category=RuntimeWarning,)
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)
    if not logger.handlers:
        handler = logging.StreamHandler()
        formatter = logging.Formatter('%(asctime)s [%(levelname)s] %(message)s')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
    mne.set_log_level("WARNING")

    def _mean_pool(data, pool_size=5):
        """
        data: (n_channels, n_times)
        Returns data_pooled: (n_channels, n_times // pool_size)
        """
        n_channels, n_times = data.shape
        remainder = n_times % pool_size
        if remainder != 0:
            data = data[:, : (n_times - remainder)]
        data = data.reshape(n_channels, -1, pool_size)
        data_pooled = data.mean(axis=-1)
        return data_pooled

    def _chunk_into_windows(data, window_size=275):
        """
        data: (n_channels, n_times_pooled)
        Returns windows: (n_windows, n_channels, window_size)
        """
        n_channels, n_times = data.shape
        n_windows = n_times // window_size
        usable = n_windows * window_size
        data = data[:, :usable]
        data = data.reshape(n_channels, n_windows, window_size)
        data = np.transpose(data, (1, 0, 2))
        return data

    def _pool_chunk_and_overwrite(fif_path, pool_size=5, window_size=275):
        """
        Process and overwrite a single .fif file with error handling
        Returns: True if successful, False if failed
        """
        try:
            # Load data
            logger.info(f"Processing & Overwriting: {fif_path}")
            raw_original = mne.io.read_raw_fif(fif_path, preload=True)
            original_data = raw_original.get_data()
            original_sf = raw_original.info['sfreq']
        except Exception as e:
            logger.error(f"Failed to read {fif_path}")
            logger.error(f"Error: {str(e)}")
            return False

        try:
            # Mean pool
            data_pooled = _mean_pool(original_data, pool_size=pool_size)
            
            # Break into windows
            windows = _chunk_into_windows(data_pooled, window_size=window_size)
            n_windows = windows.shape[0]

            # Flatten windows
            if n_windows > 0:
                flattened_data = windows.transpose(1, 0, 2).reshape(original_data.shape[0], -1)
            else:
                flattened_data = np.zeros((original_data.shape[0], 0), dtype=np.float32)

            logger.info(
                f"  Shapes:\n"
                f"    Original: {original_data.shape},\n"
                f"    After pooling: {data_pooled.shape},\n"
                f"    #windows of {window_size} frames: {n_windows},\n"
                f"    Final 'flattened' shape: {flattened_data.shape}."
            )

            ch_names = raw_original.ch_names
            ch_types = raw_original.get_channel_types()
            new_info = mne.create_info(
                ch_names=ch_names, 
                sfreq=(original_sf / pool_size), 
                ch_types=ch_types
            )
            
            new_raw = mne.io.RawArray(flattened_data, new_info)
            new_raw.set_meas_date(raw_original.info['meas_date'])
            new_raw.info['bads'] = list(raw_original.info['bads'])

            new_raw.save(fif_path, overwrite=True)
            return True

        except Exception as e:
            logger.error(f"Error processing {fif_path}")
            logger.error(f"Error: {str(e)}")
            return False

    def _process_all(dataset_path="/srv/synaptech_openmri_1"):
        """
        Process all files with error tracking
        """
        processed_files = []
        failed_files = []
        
        for mode in ["train", "val", "test"]:
            mode_path = os.path.join(dataset_path, mode)
            if not os.path.isdir(mode_path):
                logger.warning(f"Skipping non-existent folder: {mode_path}")
                continue

            subjects = sorted(os.listdir(mode_path))
            logger.info(f"[{mode}] Found {len(subjects)} potential items in: {mode_path}")

            for subject in subjects:
                subject_path = os.path.join(mode_path, subject)
                if not os.path.isdir(subject_path):
                    continue

                fif_files = [f for f in os.listdir(subject_path) if f.endswith('.fif')]
                if not fif_files:
                    logger.info(f"No .fif files found for subject {subject}, skipping.")
                    continue

                logger.info(f"Subject {subject} has {len(fif_files)} .fif runs.")
                for run_file in fif_files:
                    fif_path = os.path.join(subject_path, run_file)
                    if _pool_chunk_and_overwrite(fif_path, pool_size=5, window_size=275):
                        processed_files.append(fif_path)
                    else:
                        failed_files.append(fif_path)

        pooled_marker = os.path.join(dataset_path, '.pooled')
        with open(pooled_marker, 'w') as marker_file:
            marker_file.write('')
        
        if failed_files:
            logger.warning(f"Processing completed with {len(failed_files)} corrupted files skipped")
        print("GREAT SUCCESS")

    _process_all(datasetPath)

_meanPoolData("/home/gleb/Synaptech/data_magic/data/openfmri")


- making shards

In [ ]:
import os
import logging
import warnings
import mne
import numpy as np
import torch
from tqdm import tqdm

warnings.filterwarnings(
    "ignore",
    message=".* does not conform to MNE naming conventions.*",
    category=RuntimeWarning,
)

# ---------------------------------------------------------------------------
# Configure Python logger
# ---------------------------------------------------------------------------
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
if not logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s [%(levelname)s] %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)

# ---------------------------------------------------------------------------
# MNE logging
# ---------------------------------------------------------------------------
mne.set_log_level("WARNING")

def safe_read_fif(fif_path):
    """
    Safely read a .fif file, returning None if corrupted
    """
    try:
        raw = mne.io.read_raw_fif(fif_path, preload=True)
        return raw
    except Exception as e:
        logger.error(f"Failed to read {fif_path}: {str(e)}")
        return None

def collect_global_stats(dataset_path="/srv/synaptech_openfmri"):
    """
    Pass 1:
      - Traverse all train/val/test .fif files,
      - Collect sums and sums_of_squares for each channel (EEG & MAG).
      - Then compute mean & std for each channel.

    Returns a dictionary with:
      {
        "eeg_mean": np.array of shape [n_eeg_channels],
        "eeg_std":  np.array of shape [n_eeg_channels],
        "mag_mean": np.array of shape [n_mag_channels],
        "mag_std":  np.array of shape [n_mag_channels]
      }
    """
    logger.info("Collecting global channel stats (EEG & MAG) across the entire dataset...")

    # Track processed and failed files
    processed_files = []
    failed_files = []

    # Initialize accumulators
    eeg_channel_count = None
    mag_channel_count = None
    sums_eeg = None
    sums_sqr_eeg = None
    num_samples_eeg = 0
    sums_mag = None
    sums_sqr_mag = None
    num_samples_mag = 0

    # Potential subfolders
    for mode in ["train", "val", "test"]:
        mode_path = os.path.join(dataset_path, mode)
        if not os.path.isdir(mode_path):
            continue

        subjects = sorted(os.listdir(mode_path))
        for subject in subjects:
            subject_path = os.path.join(mode_path, subject)
            if not os.path.isdir(subject_path):
                continue

            run_files = [f for f in os.listdir(subject_path) if f.endswith(".fif")]
            for run_file in run_files:
                run_path = os.path.join(subject_path, run_file)
                logger.debug(f"(Stats) Loading {run_path}")

                # Safely read the file
                raw = safe_read_fif(run_path)
                if raw is None:
                    failed_files.append(run_path)
                    continue

                try:
                    data_all = raw.get_data()

                    # EEG
                    eeg_indices = mne.pick_types(raw.info, meg=False, eeg=True)
                    if len(eeg_indices) > 0:
                        eeg_data = data_all[eeg_indices, :]
                        n_eeg_ch, n_times = eeg_data.shape

                        if eeg_channel_count is None:
                            eeg_channel_count = n_eeg_ch
                            sums_eeg = np.zeros(eeg_channel_count, dtype=np.float64)
                            sums_sqr_eeg = np.zeros(eeg_channel_count, dtype=np.float64)

                        if n_eeg_ch != eeg_channel_count:
                            logger.error(f"EEG channel count mismatch: {n_eeg_ch} vs. {eeg_channel_count}")
                            continue

                        sums_eeg += np.sum(eeg_data, axis=1)
                        sums_sqr_eeg += np.sum(eeg_data**2, axis=1)
                        num_samples_eeg += n_times

                    # MAG
                    mag_indices = mne.pick_types(raw.info, meg='mag', eeg=False)
                    if len(mag_indices) > 0:
                        mag_data = data_all[mag_indices, :]
                        n_mag_ch, n_times = mag_data.shape

                        if mag_channel_count is None:
                            mag_channel_count = n_mag_ch
                            sums_mag = np.zeros(mag_channel_count, dtype=np.float64)
                            sums_sqr_mag = np.zeros(mag_channel_count, dtype=np.float64)

                        if n_mag_ch != mag_channel_count:
                            logger.error(f"MAG channel count mismatch: {n_mag_ch} vs. {mag_channel_count}")
                            continue

                        sums_mag += np.sum(mag_data, axis=1)
                        sums_sqr_mag += np.sum(mag_data**2, axis=1)
                        num_samples_mag += n_times

                    processed_files.append(run_path)

                except Exception as e:
                    logger.error(f"Error processing {run_path}: {str(e)}")
                    failed_files.append(run_path)
                    continue

    # Compute statistics
    if eeg_channel_count is not None and num_samples_eeg > 0:
        mean_eeg = sums_eeg / num_samples_eeg
        e_x2 = sums_sqr_eeg / num_samples_eeg
        var_eeg = e_x2 - mean_eeg**2
        var_eeg = np.maximum(var_eeg, 1e-20)
        std_eeg = np.sqrt(var_eeg)
    else:
        mean_eeg = np.array([])
        std_eeg = np.array([])

    if mag_channel_count is not None and num_samples_mag > 0:
        mean_mag = sums_mag / num_samples_mag
        e_x2 = sums_sqr_mag / num_samples_mag
        var_mag = e_x2 - mean_mag**2
        var_mag = np.maximum(var_mag, 1e-20)
        std_mag = np.sqrt(var_mag)
    else:
        mean_mag = np.array([])
        std_mag = np.array([])

    # Log summary
    logger.info(f"Stats collection complete. Processed {len(processed_files)} files.")
    if failed_files:
        logger.warning(f"Failed to process {len(failed_files)} files:")
        for f in failed_files:
            logger.warning(f"  - {f}")

    return {
        "eeg_mean": mean_eeg,
        "eeg_std": std_eeg,
        "mag_mean": mean_mag,
        "mag_std": std_mag,
        "processed_files": processed_files,
        "failed_files": failed_files
    }

def make_3d_windows(data, window_size=275, allow_padding=False, mode="EEG"):
    """
    Same as before: chunk data -> [n_channels, window_size, n_windows].
    """
    n_channels, n_times = data.shape
    n_windows = n_times // window_size
    leftover = n_times % window_size

    if n_windows == 0:
        if allow_padding and (n_times > 0):
            pad_window = np.zeros((n_channels, window_size), dtype=data.dtype)
            pad_window[:, :n_times] = data
            pad_window = pad_window.reshape(n_channels, window_size, 1)
            return torch.from_numpy(pad_window)
        else:
            logger.warning(f"Skipping {mode.upper()} because it has only {n_times} < {window_size} samples.")
            return None

    used = n_windows * window_size
    blocks = data[:, :used]
    blocks = blocks.reshape(n_channels, n_windows, window_size)
    blocks = np.transpose(blocks, (0, 2, 1))

    if leftover > 0 and allow_padding:
        pad_window = np.zeros((n_channels, window_size), dtype=data.dtype)
        pad_window[:, :leftover] = data[:, used:]
        pad_window = pad_window.reshape(n_channels, window_size, 1)
        blocks = np.concatenate([blocks, pad_window], axis=2)

    return torch.from_numpy(blocks)

def apply_zscore_and_save_shards(dataset_path, stats, window_size=275, shard_output_dir=None, allow_padding=False):
    """
    Pass 2:
      - Use the global stats (stats['eeg_mean'], stats['eeg_std'], etc.).
      - For each run, pick EEG => (data - mean) / std => chunk => save float16.
      - Same for MAG.
    """
    logger.info(f"Applying z-score normalization & saving shards in float16. Using window_size={window_size}")

    # Unpack stats
    mean_eeg = stats["eeg_mean"]
    std_eeg = stats["eeg_std"]
    mean_mag = stats["mag_mean"]
    std_mag = stats["mag_std"]

    # Track processed and failed files
    processed_files = []
    failed_files = []

    for mode in ["train", "val", "test"]:
        mode_path = os.path.join(dataset_path, mode)
        if not os.path.isdir(mode_path):
            continue

        subjects = sorted(os.listdir(mode_path))
        for subject in subjects:
            subject_path = os.path.join(mode_path, subject)
            if not os.path.isdir(subject_path):
                continue

            run_files = [f for f in os.listdir(subject_path) if f.endswith(".fif")]
            for run_file in run_files:
                run_path = os.path.join(subject_path, run_file)
                logger.info(f"[Z-score pass] Loading {run_path}")

                # Skip if file was previously identified as corrupted
                if run_path in stats.get('failed_files', []):
                    logger.warning(f"Skipping previously identified corrupted file: {run_path}")
                    failed_files.append(run_path)
                    continue

                # Safely read the file
                raw = safe_read_fif(run_path)
                if raw is None:
                    failed_files.append(run_path)
                    continue

                try:
                    data_all = raw.get_data()

                    # EEG
                    eeg_indices = mne.pick_types(raw.info, meg=False, eeg=True)
                    eeg_data = data_all[eeg_indices, :]
                    if eeg_data.size > 0 and len(eeg_indices) == len(mean_eeg):
                        for ch_i in range(len(eeg_indices)):
                            eeg_data[ch_i, :] = (eeg_data[ch_i, :] - mean_eeg[ch_i]) / std_eeg[ch_i]

                        shard_eeg = make_3d_windows(eeg_data, window_size=window_size, 
                                                  allow_padding=allow_padding, mode="EEG")
                        if shard_eeg is not None:
                            if shard_output_dir:
                                out_dir = os.path.join(shard_output_dir, mode, subject)
                            else:
                                out_dir = os.path.join(subject_path, "EEG_shards")
                            os.makedirs(out_dir, exist_ok=True)

                            name_base, _ = os.path.splitext(run_file)
                            out_fname = f"{name_base}_eeg.pt"
                            out_path = os.path.join(out_dir, out_fname)

                            shard_eeg_fp16 = shard_eeg.half()
                            torch.save(shard_eeg_fp16, out_path)
                            logger.info(f"Z-scored EEG => shape={tuple(shard_eeg_fp16.shape)}: {out_path}")

                    # MAG
                    mag_indices = mne.pick_types(raw.info, meg='mag', eeg=False)
                    mag_data = data_all[mag_indices, :]
                    if mag_data.size > 0 and len(mag_indices) == len(mean_mag):
                        for ch_i in range(len(mag_indices)):
                            mag_data[ch_i, :] = (mag_data[ch_i, :] - mean_mag[ch_i]) / std_mag[ch_i]

                        shard_mag = make_3d_windows(mag_data, window_size=window_size, 
                                                  allow_padding=allow_padding, mode="MAG")
                        if shard_mag is not None:
                            if shard_output_dir:
                                out_dir = os.path.join(shard_output_dir, mode, subject)
                            else:
                                out_dir = os.path.join(subject_path, "MAG_shards")
                            os.makedirs(out_dir, exist_ok=True)

                            name_base, _ = os.path.splitext(run_file)
                            out_fname = f"{name_base}_mag.pt"
                            out_path = os.path.join(out_dir, out_fname)

                            shard_mag_fp16 = shard_mag.half()
                            torch.save(shard_mag_fp16, out_path)
                            logger.info(f"Z-scored MAG => shape={tuple(shard_mag_fp16.shape)}: {out_path}")

                    processed_files.append(run_path)

                except Exception as e:
                    logger.error(f"Error processing {run_path}: {str(e)}")
                    failed_files.append(run_path)
                    continue

    # Log summary
    logger.info(f"\nZ-score processing complete. Processed {len(processed_files)} files.")
    if failed_files:
        logger.warning(f"Failed to process {len(failed_files)} files:")
        for f in failed_files:
            logger.warning(f"  - {f}")

    return {
        'processed': processed_files,
        'failed': failed_files
    }

def main_zscore_shard_pipeline(dataset_path="/srv/synaptech_openfmri", window_size=275, shard_output_dir=None):
    """
    1) Gather global stats (means & std) for each EEG & MAG channel across entire dataset.
    2) Re-loop, apply z-score, chunk, save shards in float16.
    """
    logger.info("=== PASS 1: Gathering global stats ===")
    stats = collect_global_stats(dataset_path=dataset_path)
    logger.info("Computed global EEG means/stds shapes: "
                f"{stats['eeg_mean'].shape}, {stats['eeg_std'].shape}")
    logger.info("Computed global MAG means/stds shapes: "
                f"{stats['mag_mean'].shape}, {stats['mag_std'].shape}")

    logger.info("=== PASS 2: Applying z-score & saving shards ===")
    results = apply_zscore_and_save_shards(
        dataset_path=dataset_path,
        stats=stats,
        window_size=window_size,
        shard_output_dir=shard_output_dir,
        allow_padding=False
    )
    
    logger.info("Z-score pipeline complete.")
    return results

if __name__ == "__main__":
    results = main_zscore_shard_pipeline(
        dataset_path="/home/gleb/Synaptech/data_magic/data/openfmri",
        window_size=275,
        shard_output_dir=None  # or e.g. "shards_zscore"
    )

### Cleanup
- removing shard folders

In [11]:
import os
import shutil
from pathlib import Path
import logging

# Set up logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
if not logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s [%(levelname)s] %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)

def cleanup_shard_folders(dataset_path="/home/gleb/Synaptech/data_magic/data/openfmri"):
    """
    Remove all 'EEG_shard' and 'MAG_shard' folders from the dataset.
    """
    logger.info(f"Starting cleanup of shard folders in: {dataset_path}")
    
    folders_removed = 0
    
    # Walk through train/val/test folders
    for mode in ["train", "val", "test"]:
        mode_path = Path(dataset_path) / mode
        if not mode_path.is_dir():
            logger.warning(f"Skipping non-existent folder: {mode_path}")
            continue
            
        # For each subject folder
        for subject_path in mode_path.iterdir():
            if not subject_path.is_dir():
                continue
                
            # Check for and remove EEG_shard folder
            eeg_shard_path = subject_path / "EEG_shards"
            if eeg_shard_path.exists():
                logger.info(f"Removing: {eeg_shard_path}")
                shutil.rmtree(eeg_shard_path)
                folders_removed += 1
                
            # Check for and remove MAG_shard folder
            mag_shard_path = subject_path / "MAG_shards"
            if mag_shard_path.exists():
                logger.info(f"Removing: {mag_shard_path}")
                shutil.rmtree(mag_shard_path)
                folders_removed += 1
    
    logger.info(f"Cleanup completed! Removed {folders_removed} shard folders.")

if __name__ == "__main__":
    cleanup_shard_folders()

2024-12-25 15:20:51,835 [INFO] Starting cleanup of shard folders in: /home/gleb/Synaptech/data_magic/data/openfmri
2024-12-25 15:20:51,836 [INFO] Cleanup completed! Removed 0 shard folders.


# Observability & Debugging

In [15]:
import torch
import os
from pathlib import Path
import logging

# ---------------------------------------------------------------------------
# Configure Python logger
# ---------------------------------------------------------------------------
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)  # Adjust to DEBUG for more verbosity
if not logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s [%(levelname)s] %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)

# ---------------------------------------------------------------------------
# Inspection Function
# ---------------------------------------------------------------------------
def inspect_all_shards(dataset_path="/srv/synaptech_openfmri", per_channel_stats=True):
    """
    Inspect all EEG and MAG shards across all modes and subjects.
    Prints global and per-channel statistics to verify z-score normalization.
    """
    logger.info(f"Starting inspection of all shards in dataset: {dataset_path}")
    
    modes = ["train", "val", "test"]
    shard_types = ["EEG_shards", "MAG_shards"]
    
    for mode in modes:
        mode_path = Path(dataset_path) / mode
        if not mode_path.is_dir():
            logger.warning(f"Skipping non-existent mode folder: {mode_path}")
            continue
        
        subjects = sorted(mode_path.iterdir())
        logger.info(f"Inspecting mode: {mode} with {len(subjects)} subjects.")
        
        for subject_path in subjects:
            if not subject_path.is_dir():
                logger.debug(f"Skipping non-directory: {subject_path}")
                continue
            
            for shard_type in shard_types:
                shard_dir = subject_path / shard_type
                if not shard_dir.is_dir():
                    logger.warning(f"Missing shard directory: {shard_dir}, skipping.")
                    continue
                
                shard_files = sorted(shard_dir.glob("*.pt"))
                logger.info(f"Inspecting {len(shard_files)} shards in {shard_dir}")
                
                for shard_file in shard_files:
                    try:
                        # Suppress the FutureWarning by specifying weights_only=True if appropriate
                        # Since these are custom shards, ensure you are loading them securely
                        tensor = torch.load(shard_file, weights_only=True)
                    except TypeError:
                        # If weights_only is not a valid argument (older PyTorch versions), load normally
                        tensor = torch.load(shard_file)
                    
                    print(f"\n{mode} | {subject_path.name} | {shard_type} | {shard_file.name}:")
                    print(f"  Shape: {tensor.shape}")
                    print(f"  Dtype: {tensor.dtype}")
                    print(f"  Device: {tensor.device}")
    
                    with torch.no_grad():
                        # Convert to float32 for accurate statistics without altering the saved data
                        data_f32 = tensor.float()
                        
                        global_min = data_f32.min().item()
                        global_max = data_f32.max().item()
                        global_mean = data_f32.mean().item()
                        global_std = data_f32.std().item()
                    
                    print(f"  (Global) Min:  {global_min:.6e}")
                    print(f"  (Global) Max:  {global_max:.6e}")
                    print(f"  (Global) Mean: {global_mean:.6e}")
                    print(f"  (Global) Std:  {global_std:.6e}")
    
                    # Optional per-channel stats
                    if per_channel_stats:
                        n_channels = data_f32.shape[0]
                        # Reshape to (n_channels, -1) without altering memory layout
                        data_2d = data_f32.reshape(n_channels, -1)
                        
                        print("  Per-channel stats:")
                        for ch_idx in range(n_channels):
                            ch_data = data_2d[ch_idx]
                            ch_min = ch_data.min().item()
                            ch_max = ch_data.max().item()
                            ch_mean = ch_data.mean().item()
                            ch_std = ch_data.std().item()
                            print(f"    Channel {ch_idx:02d}: "
                                  f"Min={ch_min:.4e}, Max={ch_max:.4e}, "
                                  f"Mean={ch_mean:.4e}, Std={ch_std:.4e}")
    
    logger.info("Completed inspection of all shards.")

# ---------------------------------------------------------------------------
# Main Execution
# ---------------------------------------------------------------------------
if __name__ == "__main__":
    inspect_all_shards(
        dataset_path="/home/gleb/Synaptech/data_magic/data/openfmri",
        per_channel_stats=True  # Set to False to skip per-channel statistics
    )


2024-12-25 17:42:37,952 [INFO] Starting inspection of all shards in dataset: /home/gleb/Synaptech/data_magic/data/openfmri
2024-12-25 17:42:37,953 [INFO] Inspecting mode: train with 11 subjects.
2024-12-25 17:42:37,953 [INFO] Inspecting 6 shards in /home/gleb/Synaptech/data_magic/data/openfmri/train/sub-01/EEG_shards
2024-12-25 17:42:38,038 [INFO] Inspecting 6 shards in /home/gleb/Synaptech/data_magic/data/openfmri/train/sub-01/MAG_shards



train | sub-01 | EEG_shards | run_01_eeg.pt:
  Shape: torch.Size([74, 275, 392])
  Dtype: torch.float16
  Device: cpu
  (Global) Min:  -5.566406e+00
  (Global) Max:  1.887500e+01
  (Global) Mean: 1.066908e-01
  (Global) Std:  5.899308e-01
  Per-channel stats:
    Channel 00: Min=-2.4395e+00, Max=2.6094e+00, Mean=1.6758e-01, Std=5.0637e-01
    Channel 01: Min=-2.5488e+00, Max=6.0938e+00, Mean=1.3312e-01, Std=7.5268e-01
    Channel 02: Min=-2.6133e+00, Max=2.5859e+00, Mean=-2.5484e-02, Std=4.6468e-01
    Channel 03: Min=-2.4414e+00, Max=5.8203e+00, Mean=4.2418e-02, Std=6.8575e-01
    Channel 04: Min=-2.7949e+00, Max=5.8945e+00, Mean=7.2771e-02, Std=7.2296e-01
    Channel 05: Min=-2.5742e+00, Max=4.9844e+00, Mean=4.0112e-02, Std=6.0379e-01
    Channel 06: Min=-2.2734e+00, Max=4.9883e+00, Mean=1.0662e-01, Std=6.2764e-01
    Channel 07: Min=-2.0059e+00, Max=5.2305e+00, Mean=1.0544e-01, Std=5.4675e-01
    Channel 08: Min=-2.0625e+00, Max=4.0586e+00, Mean=5.8997e-02, Std=4.9277e-01
    Chann

2024-12-25 17:42:38,214 [INFO] Inspecting 6 shards in /home/gleb/Synaptech/data_magic/data/openfmri/train/sub-02/EEG_shards
2024-12-25 17:42:38,280 [INFO] Inspecting 6 shards in /home/gleb/Synaptech/data_magic/data/openfmri/train/sub-02/MAG_shards


  (Global) Min:  -2.646484e-01
  (Global) Max:  2.678223e-01
  (Global) Mean: 2.374791e-03
  (Global) Std:  1.047435e-02
  Per-channel stats:
    Channel 00: Min=-5.0690e-02, Max=2.8763e-02, Mean=-2.7085e-03, Std=7.3061e-03
    Channel 01: Min=-4.4678e-02, Max=2.0493e-02, Mean=-9.0929e-03, Std=8.1224e-03
    Channel 02: Min=-4.8706e-02, Max=3.1586e-02, Mean=-4.8308e-03, Std=6.2342e-03
    Channel 03: Min=-4.9286e-02, Max=4.0039e-02, Mean=-2.8477e-03, Std=6.4872e-03
    Channel 04: Min=-5.2551e-02, Max=5.6091e-02, Mean=3.4017e-03, Std=5.5450e-03
    Channel 05: Min=-4.9500e-02, Max=2.7237e-02, Mean=-1.4425e-03, Std=5.0084e-03
    Channel 06: Min=-3.9093e-02, Max=5.3223e-02, Mean=-1.6411e-04, Std=5.4329e-03
    Channel 07: Min=-3.2959e-02, Max=4.2725e-02, Mean=-2.2684e-03, Std=6.0843e-03
    Channel 08: Min=-4.9438e-02, Max=1.5793e-02, Mean=-9.0433e-03, Std=8.4431e-03
    Channel 09: Min=-5.2216e-02, Max=3.8116e-02, Mean=-7.9240e-03, Std=7.0843e-03
    Channel 10: Min=-6.0120e-02, Max=2.

2024-12-25 17:42:38,436 [INFO] Inspecting 6 shards in /home/gleb/Synaptech/data_magic/data/openfmri/train/sub-04/EEG_shards
2024-12-25 17:42:38,501 [INFO] Inspecting 6 shards in /home/gleb/Synaptech/data_magic/data/openfmri/train/sub-04/MAG_shards


...

  (Global) Min:  -6.335938e+00
  (Global) Max:  5.000000e+00
  (Global) Mean: -5.223572e-02
  (Global) Std:  7.762591e-01
  Per-channel stats:
    Channel 00: Min=-4.5234e+00, Max=4.0391e+00, Mean=-1.2306e-01, Std=8.0861e-01
    Channel 01: Min=-1.7139e+00, Max=4.9336e+00, Mean=3.7638e-02, Std=3.8000e-01
    Channel 02: Min=-3.9121e+00, Max=3.1543e+00, Mean=-8.0526e-02, Std=6.7338e-01
    Channel 03: Min=-1.6572e+00, Max=4.6016e+00, Mean=-1.1181e-02, Std=3.5646e-01
    Channel 04: Min=-2.4375e+00, Max=5.0000e+00, Mean=-2.7159e-02, Std=4.8642e-01
    Channel 05: Min=-2.4004e+00, Max=4.5430e+00, Mean=3.6126e-03, Std=4.7654e-01
    Channel 06: Min=-2.5977e+00, Max=4.7539e+00, Mean=-2.9033e-02, Std=5.3465e-01
    Channel 07: Min=-2.0586e+00, Max=4.6680e+00, Mean=-5.4866e-03, Std=4.2767e-01
    Channel 08: Min=-2.7070e+00, Max=4.4453e+00, Mean=2.3759e-02, Std=5.0491e-01
    Channel 09: Min=-2.7109e+00, Max=4.5469e+00, Mean=-1.5977e-02, Std=5.9960e-01
    Channel 10: Min=-3.2598e+00, Max=4.6

2024-12-25 17:42:41,283 [INFO] Inspecting 6 shards in /home/gleb/Synaptech/data_magic/data/openfmri/test/sub-14/MAG_shards



test | sub-14 | EEG_shards | run_03_eeg.pt:
  Shape: torch.Size([74, 275, 415])
  Dtype: torch.float16
  Device: cpu
  (Global) Min:  -1.126562e+01
  (Global) Max:  8.453125e+00
  (Global) Mean: 3.075518e-02
  (Global) Std:  6.778036e-01
  Per-channel stats:
    Channel 00: Min=-4.8008e+00, Max=2.7773e+00, Mean=8.8488e-02, Std=6.2076e-01
    Channel 01: Min=-4.3203e+00, Max=6.7422e+00, Mean=7.9181e-02, Std=9.0272e-01
    Channel 02: Min=-5.1875e+00, Max=2.7676e+00, Mean=1.0596e-01, Std=6.4406e-01
    Channel 03: Min=-5.7930e+00, Max=6.1367e+00, Mean=-1.8361e-04, Std=6.7871e-01
    Channel 04: Min=-4.0898e+00, Max=6.3750e+00, Mean=-1.6028e-02, Std=7.6840e-01
    Channel 05: Min=-3.9727e+00, Max=5.8477e+00, Mean=1.6505e-02, Std=7.9260e-01
    Channel 06: Min=-6.1484e+00, Max=6.1523e+00, Mean=2.1493e-02, Std=8.9141e-01
    Channel 07: Min=-6.6641e+00, Max=6.2344e+00, Mean=5.4355e-02, Std=8.9932e-01
    Channel 08: Min=-4.2695e+00, Max=6.3984e+00, Mean=8.0607e-03, Std=6.5044e-01
    Chann

2024-12-25 17:42:41,454 [INFO] Completed inspection of all shards.


    Channel 11: Min=-5.5634e-02, Max=4.2236e-02, Mean=-4.4946e-03, Std=7.6656e-03
    Channel 12: Min=-4.9683e-02, Max=3.4515e-02, Mean=-6.5382e-03, Std=4.3828e-03
    Channel 13: Min=-1.3257e-01, Max=1.1230e-01, Mean=-2.2625e-03, Std=3.9141e-03
    Channel 14: Min=-6.1249e-02, Max=5.6824e-02, Mean=3.6221e-03, Std=4.0333e-03
    Channel 15: Min=-2.3706e-01, Max=2.1484e-01, Mean=-3.2655e-03, Std=4.4733e-03
    Channel 16: Min=-8.9111e-02, Max=5.3406e-02, Mean=-2.0536e-02, Std=5.9610e-03
    Channel 17: Min=-8.2825e-02, Max=5.4901e-02, Mean=-2.1582e-02, Std=6.0584e-03
    Channel 18: Min=-8.5815e-02, Max=6.7139e-02, Mean=-1.2099e-02, Std=4.2813e-03
    Channel 19: Min=-1.1115e-01, Max=6.8237e-02, Mean=-1.5409e-02, Std=4.8731e-03
    Channel 20: Min=-6.4941e-02, Max=4.8157e-02, Mean=-5.0387e-03, Std=4.0240e-03
    Channel 21: Min=-1.0907e-01, Max=1.0712e-01, Mean=1.4988e-02, Std=3.9592e-03
    Channel 22: Min=-2.6611e-01, Max=2.5122e-01, Mean=7.2652e-04, Std=5.4874e-03
    Channel 23: Min

In [18]:
eeg_data = raw_data.get_data(picks='eeg')
meg_data = raw_data.get_data(picks='meg')
print("Shape of eeg_data:", eeg_data.shape)
print("Shape of meg_data:", meg_data.shape)

print("Type of eeg_data:", type(eeg_data))
print("Type of meg_data:", type(meg_data))

print(eeg_data.nbytes / (1000**3), "GB")
print(meg_data.nbytes / (1000**3), "GB")
total_data_size_gb = (eeg_data.nbytes + meg_data.nbytes) / (1000**3)
print("Total data size for participant 1:", total_data_size_gb, "GB")

total_participant_count = 19
estimated_total_data_size_gb = total_data_size_gb * total_participant_count
print("Estimated total openFMRI data size for all participants:", estimated_total_data_size_gb, "GB")



Shape of eeg_data: (74, 540100)
Shape of meg_data: (306, 540100)
Type of eeg_data: <class 'numpy.ndarray'>
Type of meg_data: <class 'numpy.ndarray'>
0.3197392 GB
1.3221648 GB
Total data size for participant 1: 1.641904 GB
Estimated total openFMRI data size for all participants: 31.196176 GB
